# San Antonio Hotel Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
comgas_cost = 0.60
comelec_cost = 0.0806

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.53

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI and cost/sf

sf = 189.6

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Commercial Electric Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/sat/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/sat/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 394200  rows, 44 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,A1Eclear,E,Clear,01/01 01:00:00,14.875,0.975,35493.11829,0.0,0.0,0.0,...,6.263576,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,53979.40333
1,A1Eclear,E,Clear,01/01 02:00:00,12.375,0.975,35493.11829,0.0,0.0,0.0,...,6.051370,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
2,A1Eclear,E,Clear,01/01 03:00:00,12.000,0.975,15016.31927,0.0,0.0,0.0,...,5.794059,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
3,A1Eclear,E,Clear,01/01 04:00:00,12.000,0.975,15016.31927,0.0,0.0,0.0,...,5.632373,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
4,A1Eclear,E,Clear,01/01 05:00:00,12.625,0.975,15016.31927,0.0,0.0,0.0,...,5.571512,1.0,1.0,13.0,21.830,0.0,0.000000,0,166371.5353,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,C3Wlowe,W,Lowe,12/31 20:00:00,15.125,0.975,136511.99340,0.0,0.0,0.0,...,5.024463,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,22933.84749
8756,C3Wlowe,W,Lowe,12/31 21:00:00,13.375,0.975,136511.99340,0.0,0.0,0.0,...,5.012151,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,0.00000
8757,C3Wlowe,W,Lowe,12/31 22:00:00,11.750,0.750,136511.99340,0.0,0.0,0.0,...,5.015886,1.0,1.0,12.8,21.768,0.0,0.000000,0,188544.6314,0.00000
8758,C3Wlowe,W,Lowe,12/31 23:00:00,11.000,0.750,116035.19440,0.0,0.0,0.0,...,5.241458,1.0,1.0,12.8,21.768,0.0,7449.151542,0,188544.6314,0.00000


In [10]:
# list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [12]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

In [ ]:
# # Export grouped file to csv
# all_ComElec1.to_csv("Scraper_Output/sat_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [13]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [14]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [15]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [16]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [17]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

# get_annual_ComElec

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [18]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [19]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [20]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [21]:
get_annual_ComElec["CO2_Gas(lb/sf)"] = ((get_annual_ComElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual_ComElec

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [22]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [23]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [24]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [25]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [26]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [27]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [28]:
# # Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

# allAnnualDataComElec

In [29]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [30]:
# Group by scenario
annualDataCom = allAnnualDataComElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataCom.sum().round(2)

In [31]:
annualDataComFinal = annualDataCom.sum().round(2)

annualDataComFinal

,,,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
A1Eclear,E,Clear,1381.36,67.95,0.0,4096.20,467.12,6012.63,7.29,0.36,0.0,21.60,...,0.0,96.76,467.12,596.92,0.17,0.00,0.0,0.51,2.46,3.15
A1Eexist,E,Exist,1349.58,380.67,0.0,4070.76,467.12,6268.13,7.12,2.01,0.0,21.47,...,0.0,96.16,467.12,597.44,0.17,0.01,0.0,0.51,2.46,3.15
A1Elowe,E,Lowe,1198.60,145.44,0.0,3264.34,467.12,5075.50,6.32,0.77,0.0,17.22,...,0.0,77.11,467.12,573.42,0.15,0.00,0.0,0.41,2.46,3.02
A1Nclear,N,Clear,1332.99,186.25,0.0,3151.07,467.12,5137.43,7.03,0.98,0.0,16.62,...,0.0,74.43,467.12,574.16,0.17,0.01,0.0,0.39,2.46,3.03
A1Nexist,N,Exist,1288.73,741.80,0.0,3157.01,467.12,5654.66,6.80,3.91,0.0,16.65,...,0.0,74.57,467.12,576.59,0.16,0.02,0.0,0.39,2.46,3.04
A1Nlowe,N,Lowe,1167.58,308.68,0.0,2666.33,467.12,4609.71,6.16,1.63,0.0,14.06,...,0.0,62.98,467.12,559.54,0.15,0.01,0.0,0.33,2.46,2.95
A1Wclear,W,Clear,2030.88,29.68,0.0,5065.73,467.12,7593.41,10.71,0.16,0.0,26.72,...,0.0,119.66,467.12,634.93,0.25,0.00,0.0,0.63,2.46,3.35
A1Wexist,W,Exist,2116.02,251.53,0.0,5269.30,467.12,8103.96,11.16,1.33,0.0,27.79,...,0.0,124.47,467.12,643.08,0.26,0.01,0.0,0.66,2.46,3.39
A1Wlowe,W,Lowe,1628.93,87.15,0.0,3855.52,467.12,6038.72,8.59,0.46,0.0,20.33,...,0.0,91.07,467.12,597.20,0.20,0.00,0.0,0.48,2.46,3.15


In [ ]:
# # Export grouped file to csv
# annualDataComFinal.to_csv("Scraper_Output/sat_annual_com.csv", header=True, index=True)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>


In [33]:
# # Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [58]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={'Scenario':'Scenario',
 'Date_Time':'Date_Time',
 'Elevation':'Elevation',
 'GlazingSystem':'GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

---
## Group and Export Baseline Annual Data
Group dataframe by scenario<br>
Export to file

In [60]:
# Group by scenario
annualBaselineDataCom = allBaselineAnnualDataComElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataCom.sum().round(2)

In [61]:
annualBaselineDataCom = annualBaselineDataCom.sum().round(2)

# annualDataComFinal

In [37]:
# list(AnnualWithBaselineCom.columns.values)

In [ ]:
# # Export grouped file to csv
# annualBaselineDataCom.to_csv("Scraper_Output/sat_baseline_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [40]:
# Copy dataframe to modify and leave original df intact
get_peak_ComElec = all_ComElec2.copy(deep=True)

# get_peak_ComElec

In [42]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalComPeak = (get_peak_ComElec["FanEnergy[J](Hourly)"] + get_peak_ComElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ComElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ComElec["CoolingEnergy[J](Hourly)"] + get_peak_ComElec["LightingEnergy[J](Hourly)"])
        
get_peak_ComElec["PeakEnergy[J](Hourly)"] = totalComPeak

get_peak_ComElec["PeakEnergy(w)"] = (get_peak_ComElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ComElec["PeakEnergy(w/sf)"] = (get_peak_ComElec["PeakEnergy(w)"] / sf)

# get_peak_ComElec

In [43]:
# Get scenario and find the max
A1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eexist"]
A1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eclear"]
A1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Elowe"]
A1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nexist"]
A1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nclear"]
A1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nlowe"]
A1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wexist"]
A1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wclear"]
A1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wlowe"]
A2Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sexist"]
A2Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sclear"]
A2Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Slowe"]
B1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eexist"]
B1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eclear"]
B1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Elowe"]
B1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nexist"]
B1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nclear"]
B1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nlowe"]
B1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wexist"]
B1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wclear"]
B1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wlowe"]
B2Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Eexist"]
B2Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Eclear"]
B2Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Elowe"]
B3Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sexist"]
B3Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sclear"]
B3Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Slowe"]
B3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wexist"]
B3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wclear"]
B3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wlowe"]
C1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eexist"]
C1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eclear"]
C1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Elowe"]
C1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nexist"]
C1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nclear"]
C1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nlowe"]
C1Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sexist"]
C1Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sclear"]
C1Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Slowe"]
C2Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wexist"]
C2Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wclear"]
C2Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wlowe"]
C3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wexist"]
C3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wclear"]
C3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wlowe"]

In [44]:
A1EexistPeak = (A1Eexist[A1Eexist["PeakEnergy[J](Hourly)"] == A1Eexist["PeakEnergy[J](Hourly)"].max()])
A1EclearPeak = (A1Eclear[A1Eclear["PeakEnergy[J](Hourly)"] == A1Eclear["PeakEnergy[J](Hourly)"].max()])
A1ElowePeak = (A1Elowe[A1Elowe["PeakEnergy[J](Hourly)"] == A1Elowe["PeakEnergy[J](Hourly)"].max()])
A1NexistPeak = (A1Nexist[A1Nexist["PeakEnergy[J](Hourly)"] == A1Nexist["PeakEnergy[J](Hourly)"].max()])
A1NclearPeak = (A1Nclear[A1Nclear["PeakEnergy[J](Hourly)"] == A1Nclear["PeakEnergy[J](Hourly)"].max()])
A1NlowePeak = (A1Nlowe[A1Nlowe["PeakEnergy[J](Hourly)"] == A1Nlowe["PeakEnergy[J](Hourly)"].max()])
A1WexistPeak = (A1Wexist[A1Wexist["PeakEnergy[J](Hourly)"] == A1Wexist["PeakEnergy[J](Hourly)"].max()])
A1WclearPeak = (A1Wclear[A1Wclear["PeakEnergy[J](Hourly)"] == A1Wclear["PeakEnergy[J](Hourly)"].max()])
A1WlowePeak = (A1Wlowe[A1Wlowe["PeakEnergy[J](Hourly)"] == A1Wlowe["PeakEnergy[J](Hourly)"].max()])
A2SexisttPeak = (A2Sexist[A2Sexist["PeakEnergy[J](Hourly)"] == A2Sexist["PeakEnergy[J](Hourly)"].max()])
A2SclearPeak = (A2Sclear[A2Sclear["PeakEnergy[J](Hourly)"] == A2Sclear["PeakEnergy[J](Hourly)"].max()])
A2SlowePeak = (A2Slowe[A2Slowe["PeakEnergy[J](Hourly)"] == A2Slowe["PeakEnergy[J](Hourly)"].max()])
B1EexistPeak = (B1Eexist[B1Eexist["PeakEnergy[J](Hourly)"] == B1Eexist["PeakEnergy[J](Hourly)"].max()])
B1EclearPeak = (B1Eclear[B1Eclear["PeakEnergy[J](Hourly)"] == B1Eclear["PeakEnergy[J](Hourly)"].max()])
B1ElowePeak = (B1Elowe[B1Elowe["PeakEnergy[J](Hourly)"] == B1Elowe["PeakEnergy[J](Hourly)"].max()])
B1NexistPeak = (B1Nexist[B1Nexist["PeakEnergy[J](Hourly)"] == B1Nexist["PeakEnergy[J](Hourly)"].max()])
B1NclearPeak = (B1Nclear[B1Nclear["PeakEnergy[J](Hourly)"] == B1Nclear["PeakEnergy[J](Hourly)"].max()])
B1NlowePeak = (B1Nlowe[B1Nlowe["PeakEnergy[J](Hourly)"] == B1Nlowe["PeakEnergy[J](Hourly)"].max()])
B1WexistPeak = (B1Wexist[B1Wexist["PeakEnergy[J](Hourly)"] == B1Wexist["PeakEnergy[J](Hourly)"].max()])
B1WclearPeak = (B1Wclear[B1Wclear["PeakEnergy[J](Hourly)"] == B1Wclear["PeakEnergy[J](Hourly)"].max()])
B1WlowePeak = (B1Wlowe[B1Wlowe["PeakEnergy[J](Hourly)"] == B1Wlowe["PeakEnergy[J](Hourly)"].max()])
B2EexistPeak = (B2Eexist[B2Eexist["PeakEnergy[J](Hourly)"] == B2Eexist["PeakEnergy[J](Hourly)"].max()])
B2EclearPeak = (B2Eclear[B2Eclear["PeakEnergy[J](Hourly)"] == B2Eclear["PeakEnergy[J](Hourly)"].max()])
B2ElowePeak = (B2Elowe[B2Elowe["PeakEnergy[J](Hourly)"] == B2Elowe["PeakEnergy[J](Hourly)"].max()])
B3SexistPeak = (B3Sexist[B3Sexist["PeakEnergy[J](Hourly)"] == B3Sexist["PeakEnergy[J](Hourly)"].max()])
B3SclearPeak = (B3Sclear[B3Sclear["PeakEnergy[J](Hourly)"] == B3Sclear["PeakEnergy[J](Hourly)"].max()])
B3SlowePeak = (B3Slowe[B3Slowe["PeakEnergy[J](Hourly)"] == B3Slowe["PeakEnergy[J](Hourly)"].max()])
B3WexistPeak = (B3Wexist[B3Wexist["PeakEnergy[J](Hourly)"] == B3Wexist["PeakEnergy[J](Hourly)"].max()])
B3WclearPeak = (B3Wclear[B3Wclear["PeakEnergy[J](Hourly)"] == B3Wclear["PeakEnergy[J](Hourly)"].max()])
B3WlowePeak = (B3Wlowe[B3Wlowe["PeakEnergy[J](Hourly)"] == B3Wlowe["PeakEnergy[J](Hourly)"].max()])
C1EexistPeak = (C1Eexist[C1Eexist["PeakEnergy[J](Hourly)"] == C1Eexist["PeakEnergy[J](Hourly)"].max()])
C1EclearPeak = (C1Eclear[C1Eclear["PeakEnergy[J](Hourly)"] == C1Eclear["PeakEnergy[J](Hourly)"].max()])
C1ElowePeak = (C1Elowe[C1Elowe["PeakEnergy[J](Hourly)"] == C1Elowe["PeakEnergy[J](Hourly)"].max()])
C1NexistPeak = (C1Nexist[C1Nexist["PeakEnergy[J](Hourly)"] == C1Nexist["PeakEnergy[J](Hourly)"].max()])
C1NclearPeak = (C1Nclear[C1Nclear["PeakEnergy[J](Hourly)"] == C1Nclear["PeakEnergy[J](Hourly)"].max()])
C1NlowePeak = (C1Nlowe[C1Nlowe["PeakEnergy[J](Hourly)"] == C1Nlowe["PeakEnergy[J](Hourly)"].max()])
C1SexistPeak = (C1Sexist[C1Sexist["PeakEnergy[J](Hourly)"] == C1Sexist["PeakEnergy[J](Hourly)"].max()])
C1SclearPeak = (C1Sclear[C1Sclear["PeakEnergy[J](Hourly)"] == C1Sclear["PeakEnergy[J](Hourly)"].max()])
C1SlowePeak = (C1Slowe[C1Slowe["PeakEnergy[J](Hourly)"] == C1Slowe["PeakEnergy[J](Hourly)"].max()])
C2WexistPeak = (C2Wexist[C2Wexist["PeakEnergy[J](Hourly)"] == C2Wexist["PeakEnergy[J](Hourly)"].max()])
C2WclearPeak = (C2Wclear[C2Wclear["PeakEnergy[J](Hourly)"] == C2Wclear["PeakEnergy[J](Hourly)"].max()])
C2WlowePeak = (C2Wlowe[C2Wlowe["PeakEnergy[J](Hourly)"] == C2Wlowe["PeakEnergy[J](Hourly)"].max()])
C3WexistPeak = (C3Wexist[C3Wexist["PeakEnergy[J](Hourly)"] == C3Wexist["PeakEnergy[J](Hourly)"].max()])
C3WclearPeak = (C3Wclear[C3Wclear["PeakEnergy[J](Hourly)"] == C3Wclear["PeakEnergy[J](Hourly)"].max()])
C3WlowePeak = (C3Wlowe[C3Wlowe["PeakEnergy[J](Hourly)"] == C3Wlowe["PeakEnergy[J](Hourly)"].max()])

In [46]:
PeakDemandElec = A1EexistPeak.append([A1EclearPeak,
A1ElowePeak,
A1NexistPeak,
A1NclearPeak,
A1NlowePeak,
A1WexistPeak,
A1WclearPeak,
A1WlowePeak,
A2SexisttPeak,
A2SclearPeak,
A2SlowePeak,
B1EexistPeak,
B1EclearPeak,
B1ElowePeak,
B1NexistPeak,
B1NclearPeak,
B1NlowePeak,
B1WexistPeak,
B1WclearPeak,
B1WlowePeak,
B2EexistPeak,
B2EclearPeak,
B2ElowePeak,
B3SexistPeak,
B3SclearPeak,
B3SlowePeak,
B3WexistPeak,
B3WclearPeak,
B3WlowePeak,
C1EexistPeak,
C1EclearPeak,
C1ElowePeak,
C1NexistPeak,
C1NclearPeak,
C1NlowePeak,
C1SexistPeak,
C1SclearPeak,
C1SlowePeak,
C2WexistPeak,
C2WclearPeak,
C2WlowePeak,
C3WexistPeak,
C3WclearPeak,
C3WlowePeak])

# PeakDemandElec

In [ ]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/sat_peak_com.csv", header=True, index=True)

## Find deltas from baseline
Determine the baselines (existing)<br>
Determine delta EUI, cost and carbon from baseline<br>
Export to csv

In [76]:
annualDataComFinal2 = annualDataComFinal

annualDataComFinal2

,,,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
A1Eclear,E,Clear,1381.36,67.95,0.0,4096.20,467.12,6012.63,7.29,0.36,0.0,21.60,...,0.0,96.76,467.12,596.92,0.17,0.00,0.0,0.51,2.46,3.15
A1Eexist,E,Exist,1349.58,380.67,0.0,4070.76,467.12,6268.13,7.12,2.01,0.0,21.47,...,0.0,96.16,467.12,597.44,0.17,0.01,0.0,0.51,2.46,3.15
A1Elowe,E,Lowe,1198.60,145.44,0.0,3264.34,467.12,5075.50,6.32,0.77,0.0,17.22,...,0.0,77.11,467.12,573.42,0.15,0.00,0.0,0.41,2.46,3.02
A1Nclear,N,Clear,1332.99,186.25,0.0,3151.07,467.12,5137.43,7.03,0.98,0.0,16.62,...,0.0,74.43,467.12,574.16,0.17,0.01,0.0,0.39,2.46,3.03
A1Nexist,N,Exist,1288.73,741.80,0.0,3157.01,467.12,5654.66,6.80,3.91,0.0,16.65,...,0.0,74.57,467.12,576.59,0.16,0.02,0.0,0.39,2.46,3.04
A1Nlowe,N,Lowe,1167.58,308.68,0.0,2666.33,467.12,4609.71,6.16,1.63,0.0,14.06,...,0.0,62.98,467.12,559.54,0.15,0.01,0.0,0.33,2.46,2.95
A1Wclear,W,Clear,2030.88,29.68,0.0,5065.73,467.12,7593.41,10.71,0.16,0.0,26.72,...,0.0,119.66,467.12,634.93,0.25,0.00,0.0,0.63,2.46,3.35
A1Wexist,W,Exist,2116.02,251.53,0.0,5269.30,467.12,8103.96,11.16,1.33,0.0,27.79,...,0.0,124.47,467.12,643.08,0.26,0.01,0.0,0.66,2.46,3.39
A1Wlowe,W,Lowe,1628.93,87.15,0.0,3855.52,467.12,6038.72,8.59,0.46,0.0,20.33,...,0.0,91.07,467.12,597.20,0.20,0.00,0.0,0.48,2.46,3.15


In [78]:
annualBaselineDataCom2 = annualBaselineDataCom
annualBaselineDataCom2

,,,B_FanEnergy(kBtu),B_HeatingElecEnergy(kBtu),B_HeatingGasEnergy(kBtu),B_CoolingEnergy(kBtu),B_LightingEnergy(kBtu),B_AnnualEnergy(kBtu),B_FanEnergy(kBtu/sf),B_HeatingElecEnergy(kBtu/sf),B_HeatingGasEnergy(kBtu/sf),B_CoolingEnergy(kBtu/sf),...,B_HeatingGasCost($),B_CoolingCost($),B_LightingCost($),B_AnnualCost($),B_FanCost($/sf),B_HeatingElecCost($/sf),B_HeatingGasCost($/sf),B_CoolingCost($/sf),B_LightingCost($/sf),B_AnnualCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
A1Eexist,E,Exist,1349.58,380.67,0.0,4070.76,467.12,6268.13,7.12,2.01,0.0,21.47,...,0.0,96.16,467.12,597.44,0.17,0.01,0.0,0.51,2.46,3.15
A1Nexist,N,Exist,1288.73,741.80,0.0,3157.01,467.12,5654.66,6.80,3.91,0.0,16.65,...,0.0,74.57,467.12,576.59,0.16,0.02,0.0,0.39,2.46,3.04
A1Wexist,W,Exist,2116.02,251.53,0.0,5269.30,467.12,8103.96,11.16,1.33,0.0,27.79,...,0.0,124.47,467.12,643.08,0.26,0.01,0.0,0.66,2.46,3.39
A2Sexist,S,Exist,1982.19,450.03,0.0,5332.18,467.12,8231.52,10.45,2.37,0.0,28.12,...,0.0,125.95,467.12,642.60,0.25,0.01,0.0,0.66,2.46,3.39
B1Eexist,E,Exist,1365.23,342.64,0.0,4233.16,467.12,6408.15,7.20,1.81,0.0,22.33,...,0.0,99.99,467.12,601.42,0.17,0.01,0.0,0.53,2.46,3.17
B1Nexist,N,Exist,1281.34,758.16,0.0,3116.68,467.12,5623.30,6.76,4.00,0.0,16.44,...,0.0,73.62,467.12,575.56,0.16,0.02,0.0,0.39,2.46,3.04
B1Wexist,W,Exist,1936.46,307.15,0.0,4943.07,467.12,7653.79,10.21,1.62,0.0,26.07,...,0.0,116.76,467.12,631.47,0.24,0.01,0.0,0.62,2.46,3.33
B2Eexist,E,Exist,1307.43,1164.87,0.0,3874.40,467.12,6813.82,6.90,6.14,0.0,20.43,...,0.0,91.52,467.12,596.51,0.16,0.04,0.0,0.48,2.46,3.15
B3Sexist,S,Exist,2072.12,436.79,0.0,5405.10,467.12,8381.12,10.93,2.30,0.0,28.51,...,0.0,127.68,467.12,646.37,0.26,0.01,0.0,0.67,2.46,3.41


In [51]:
# annualDataComFinal2["DeltaCost($/sf)"]  = (annualDataComFinal2[annualDataComFinal2["Scenario"] == "A1Eexist"] - annualBaselineDataCom2[annualBaselineDataCom2["B_Scenario"] == "A1Eexist"])

In [79]:
A1Eexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Eexist"]
A1Eclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Eclear"]
A1Elowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Elowe"]
A1Nexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Nexist"]
A1Nclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Nclear"]
A1Nlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Nlowe"]
A1Wexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Wexist"]
A1Wclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Wclear"]
A1Wlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A1Wlowe"]
A2Sexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A2Sexist"]
A2Sclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A2Sclear"]
A2Slowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "A2Slowe"]
B1Eexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Eexist"]
B1Eclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Eclear"]
B1Elowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Elowe"]
B1Nexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Nexist"]
B1Nclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Nclear"]
B1Nlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Nlowe"]
B1Wexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Wexist"]
B1Wclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Wclear"]
B1Wlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B1Wlowe"]
B2Eexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B2Eexist"]
B2Eclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B2Eclear"]
B2Elowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B2Elowe"]
B3Sexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Sexist"]
B3Sclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Sclear"]
B3Slowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Slowe"]
B3Wexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Wexist"]
B3Wclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Wclear"]
B3Wlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "B3Wlowe"]
C1Eexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Eexist"]
C1Eclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Eclear"]
C1Elowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Elowe"]
C1Nexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Nexist"]
C1Nclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Nclear"]
C1Nlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Nlowe"]
C1Sexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Sexist"]
C1Sclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Sclear"]
C1Slowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C1Slowe"]
C2Wexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C2Wexist"]
C2Wclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C2Wclear"]
C2Wlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C2Wlowe"]
C3Wexist = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C3Wexist"]
C3Wclear = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C3Wclear"]
C3Wlowe = annualBaselineDataCom2[annualBaselineDataCom2["Scenario"] == "C3Wlowe"]

KeyError: 'Scenario'

In [73]:
A1Eexist

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
8760,A1Eexist,01/01 01:00:00,E,Exist,12081.51888,0,162543.4886,0.000000,35493.11829,210118.125770,58.366146,0.307838
8761,A1Eexist,01/01 02:00:00,E,Exist,12561.24421,0,162543.4886,0.000000,35493.11829,210597.851100,58.499403,0.308541
8762,A1Eexist,01/01 03:00:00,E,Exist,0.00000,0,162543.4886,0.000000,15016.31927,177559.807870,49.322169,0.260138
8763,A1Eexist,01/01 04:00:00,E,Exist,0.00000,0,162543.4886,0.000000,15016.31927,177559.807870,49.322169,0.260138
8764,A1Eexist,01/01 05:00:00,E,Exist,0.00000,0,162543.4886,0.000000,15016.31927,177559.807870,49.322169,0.260138
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,A1Eexist,12/31 20:00:00,E,Exist,0.00000,0,162543.4886,90386.240000,136511.99340,389441.722000,108.178256,0.570560
17516,A1Eexist,12/31 21:00:00,E,Exist,0.00000,0,162543.4886,51606.442160,136511.99340,350661.924160,97.406090,0.513745
17517,A1Eexist,12/31 22:00:00,E,Exist,0.00000,0,162543.4886,2027.522172,136511.99340,301083.004172,83.634168,0.441108
17518,A1Eexist,12/31 23:00:00,E,Exist,12798.57682,0,162543.4886,0.000000,116035.19440,291377.259820,80.938128,0.426889
